In [ ]:
# standard setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2

# useful helper function
from helpers import imshow

In [ ]:
# setup capture
camera = cv2.VideoCapture(0)

# reduce frame size to speed it up
w = 640
camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 

# capture loop
while True:
    
    # get frame
    ret, frame = camera.read()
    
    # get frame size
    w, h = frame.shape[:2]
    
    # mirror the frame (my camera mirrors by default)
    frame = cv2.flip(frame, 1)
        

    # Draw rectange box controller for blurring
    xtop = ytop = 50
    xbottom = 400
    ybottom = 100
    cv2.rectangle(frame, (ytop, xtop), (ybottom, xbottom), (0,0,255), 2)
    
    # Extract out the rectangular box
    rect_box = frame[xtop:xbottom, ytop:ybottom, :]
    rect_box = cv2.cvtColor(rect_box, cv2.COLOR_BGR2GRAY)
    
    ret, rect_box_mask = cv2.threshold(rect_box, 120, 255, cv2.THRESH_BINARY)
    total_box_pixel_count = len(rect_box_mask) * len(rect_box_mask[0])
    masked_pixel_count = np.count_nonzero(rect_box_mask)
    
    percent_covered = int((masked_pixel_count / total_box_pixel_count) * 100)
    
    # Show percentage blurness to user
    cv2.putText(frame,str(percent_covered) + '%',(10,470), cv2.FONT_HERSHEY_SIMPLEX, 3,(255,255,255),2)    
    
    ksize = percent_covered + int(percent_covered % 2 == 0) 
    dst = cv2.GaussianBlur(frame,(ksize,ksize), 0)
    
    # Display the frame with blurness applied
    cv2.imshow('blur', dst)
    
    # exit on ESC press
    if cv2.waitKey(5) == 27:
        break
        
# clean up
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window